In [ ]:
from nglui import parser, statebuilder
import pandas as pd
import numpy as np
from scipy import interpolate

from caveclient import CAVEclient

## Generating a streamline
Building a streamline requires only a few parameters and a neuroglancer state.
The neuroglancer state should include an annotation layer with linked segmentation ids.
For several neurons, lay down points every 50 microns or so along the principle pia-to-white matter axis.
The apical dendrite and primary axon of layer 5 thick tufted cells (or ET cells) is a particularly good case, as are tall layer 6 apical dendrites.
It's important to have overlapping coverage from several neurons at all depths from layer 2 to white matter.
Stop placing points when apical tufts bifurcate, as that no longer represents the "up" direction for the neuron.
It is important that the layer use linked segmentation ids, so that the collection of points associated with each neuron can be easily separated.

### Parameters
* `datastack_name` : Caveclient datastack name.
* `state_id` : State ID to download the neuroglancer state described above.
* `streamline_layer_name` : Name of the annotation layer with the streamline points.
* `y_min`, `y_max` : Top and bottom depths through which the final streamline points should be produced. Typically, a bit above the pial surface to within white matter. The solver will linearly extrapolate beyond the actual data, but this will produce complete coverage and be a useful visual helper. Note that this should be in standard neuroglancer voxel units.
* `x_ctr`, `z_ctr` : The x, z values to use for the top of the streamline. This should be near the center of the cells you've taken, in case the streamline changes with space in your volume. Note that this should be in standard neuroglancer voxel units.
* `density`: Density of points along the y-axis. Note that it should be in standard neuroglancer voxel units.



In [ ]:
datastack_name = 'minnie65_phase3_v1'
state_id = 5912681645080576
streamline_layer_name = 'streamline'

# All values below are in neuroglancer voxel coordinates
y_min = 85_000
y_max = 280_000

x_ctr = 181000
z_ctr = 21600

density = 1_000

### Running

After setting parameters in the next cell, run all cells in this section to produce a streamline based on the average vector differences in x and z along the depth axis.

In [ ]:
client = CAVEclient(datastack_name)

state = client.state.get_state_json(state_id)

# Read in the neuroglancer state.

anno_df = parser.annotation_dataframe(state)
sl_df = anno_df.query('layer == @streamline_layer_name').reset_index(drop=True)

group = sl_df['linked_segmentation'].apply(lambda x: x[0])
points = np.vstack(sl_df['point'].values)

df = pd.DataFrame(
    {
        'pt_x' : points[:,0],
        'pt_y' : points[:,1],
        'pt_z' : points[:,2],
        'group' : group,
    }
)

grp_id = np.unique(df['group'])
grp_remap = {gid: ii for ii, gid in enumerate(grp_id)}
df['group'] = df['group'].apply(lambda x: grp_remap[x])

# Produce interpolated datapoints based on the real data in order to match depths.

all_new_dx = []
all_new_y = []
all_new_dz = []
all_new_groups = []
for ii in np.unique(df['group']):
    df_g = df.query('group == @ii').sort_values(by='pt_y')
    fx_g = interpolate.interp1d(df_g['pt_y'], df_g['pt_x'], kind='linear', bounds_error=False)
    fz_g = interpolate.interp1d(df_g['pt_y'], df_g['pt_z'], kind='linear', bounds_error=False)

    new_y = np.arange(y_min, y_max, density)
    new_x = fx_g(new_y)
    new_z = fz_g(new_y)
    del_x = np.diff(new_x)
    del_y = np.diff(new_y)
    del_z = np.diff(new_z)
    
    all_new_dx.append(del_x)
    all_new_y.append(new_y[:-1])
    all_new_dz.append(del_z)
    all_new_groups.append([ii]*(len(new_y)-1))

# Average the vector difference at each depth value and then generate a dense path 

dfi = pd.DataFrame(
    {
    'del_x': np.concatenate(all_new_dx),
    'del_z': np.concatenate(all_new_dz),
    'pt_y': np.concatenate(all_new_y),
    'group': np.concatenate(all_new_groups),
    }
).dropna()

dfi_avg = dfi.groupby('pt_y').mean()

all_pts = np.vstack(
    [x_ctr+np.cumsum(dfi_avg['del_x']),
     dfi_avg.index.values,
     z_ctr+np.cumsum(dfi_avg['del_z']),
    ]).T


# Generate a new collection of streamline points with specificty density based on
# the average trajectory of each interpolation at the center x, z value specified.

fx_all = interpolate.interp1d(
    all_pts[:, 1],
    all_pts[:, 0],
    kind="linear",
    fill_value="extrapolate",
    bounds_error=False,
)

fz_all = interpolate.interp1d(
    all_pts[:, 1],
    all_pts[:, 2],
    kind="linear",
    fill_value="extrapolate",
    bounds_error=False,
)


new_y = np.arange(y_min, y_max, density)
new_x = fx_all(new_y)
new_z = fz_all(new_y)
new_pts = np.vstack([new_x, new_y, new_z]).T

### Sanity checking

After producing your streamline, double check that it actually makes sense with neurons.
The following will create a neuroglancer link showing the full streamline points.
Look at the apical dendrites of deep layer cells and the primary axons of cells along cortical depth to make sure this matches expectations.

In [ ]:
img, seg = statebuilder.from_client(client)

ptmap = statebuilder.PointMapper()
anno = statebuilder.AnnotationLayerConfig(
    "points", array_data=True, mapping_rules=ptmap
)

sb = statebuilder.StateBuilder([img, seg, anno], client=client)

sb.render_state(new_pts, return_as="html")

### Saving a streamline

To generate a useful file for the Streamline class, the most consistent approach is to save it in the expected post-transform space that you will use when interacting with the Streamline class.
Since we have done everything in voxels, the easiest thing to do is transform by the voxel transform.
Here, we are in the Minnie65 data so we can use the prebaked transform there.
In general, however, this might not be an existing function.


In [ ]:
filename = 'minnie_streamline_um.json'

from standard_transform import minnie_transform_vx
tform = minnie_transform_vx()

In [ ]:
import json
pts_tform = tform.apply(new_pts)

with open(filename, 'w') as f:
    json.dump(
        pts_tform.tolist(),
        f,
    )